## Chapter 7: Functional Programming

This chapter covers the introduction to functional programming.  In short, functional programming languages have functions as a important part of the language and apply and compose functions to create programs.

### 7.1: Functional vs. Non-functional forms

Here is a simple example about create an array from another array in both non-functional and functional forms:

In [1]:
v=collect(1:5)

5-element Vector{Int64}:
 1
 2
 3
 4
 5

This is the traditional (non-functional form) of making a new array of squares of the original.

In [2]:
v2=zeros(Int,5) # this an array of zeros of length 5
for i=1:5
  v2[i]=v[i]^2
end
v2

5-element Vector{Int64}:
  1
  4
  9
 16
 25

Here are some functional form versions instead:

In [3]:
f(x)=x^2
map(f,v)

5-element Vector{Int64}:
  1
  4
  9
 16
 25

This is the same with an anonymous function

In [4]:
map(x->x^2,v)

5-element Vector{Int64}:
  1
  4
  9
 16
 25

In [6]:
map(x -> x^2, 1:5)

5-element Vector{Int64}:
  1
  4
  9
 16
 25

And recall that we did this in the previous chapter with:

In [5]:
v.^2

5-element Vector{Int64}:
  1
  4
  9
 16
 25

The `map` function starts with an array, applies a function to each element and returns the new array.  Array an input and array as output.


#### Exercise
create a vector from 1 to 10 and 1) make a new array of `[1, 0.5, 0.333, 0.25, ...]`  and 2) make another with `[1//1 1//2 1//3 1//4 ...`] using the `map` function and the anonymous function notation.  Can you also get the same result with broadcasting?

In [7]:
map(x -> 1/x, 1:10)

10-element Vector{Float64}:
 1.0
 0.5
 0.3333333333333333
 0.25
 0.2
 0.16666666666666666
 0.14285714285714285
 0.125
 0.1111111111111111
 0.1

In [10]:
arr = 1 .// collect(1:10)

10-element Vector{Rational{Int64}}:
  1
 1//2
 1//3
 1//4
 1//5
 1//6
 1//7
 1//8
 1//9
 1//10

In [11]:
typeof(arr[1])

Rational{Int64}

### 7.2: Reducing an array

Another very common idea with arrays is to start with an array and reduce it to a single number.  The classic example is a sum. 

In [12]:
A=[1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

To do this, we use the `reduce` function, which has the same form as the `map` function except the function has two arguments. 

In [13]:
reduce((x,y)-> x+y, A)

15

To see really what's going on, here's a debugging version of that:

In [14]:
function f(tot,val)
  @show tot,val
  tot+val
end

f (generic function with 2 methods)

In [15]:
reduce(f,A)

(tot, val) = (1, 2)
(tot, val) = (3, 3)
(tot, val) = (6, 4)
(tot, val) = (10, 5)


15

Notice that the first variable keeps the running sum.  Also, notice that the first call, when `val=1` is missing.  This is because the first value needs the function applied to something (it's binary).  If you don't put in an initial value, (often) it is initialized to 0 and a sum is applied on the first step.  Here's more what's going on.

In [16]:
reduce(f,A,init=0)

(tot, val) = (0, 1)
(tot, val) = (1, 2)
(tot, val) = (3, 3)
(tot, val) = (6, 4)
(tot, val) = (10, 5)


15

In [17]:
reduce((x,y)->x*y,A)

120

In [18]:
reduce(+, A)

15

In [19]:
reduce(*, A)

120

If we have an array that we want to count the number of position element in the array, then here's a nice way to do this with `reduce`.  Again, this array as an input, value as an output.

In [20]:
numPos(arr::Vector{Int64}) = reduce((num,val) -> val > 0 ? num+1 : num, arr, init=0)

numPos (generic function with 1 method)

In [21]:
numPos([-3,5,8,-2,11])

3

#### Exercise

Use reduce to take an array of strings, say `["The","dog","bit","the","cat"]` and concatenate all of the strings. (Can you figure out how to add a space between words?)

In [4]:
reduce( (str, val) -> str * val  , ["The","dog","bit","the","cat"])
reduce( *, ["The","dog","bit","the","cat"])
reduce( (str, val) -> str * " " * val  , ["The","dog","bit","the","cat"])
reduce( (str, val) -> "$str $val"  , ["The","dog","bit","the","cat"])

"The dog bit the cat"

#### Summary of reduce

If you have an 1D array (vector) of numbers/strings (or anything) and you want to summarize the vector with a single value (number, string), you can often use reduced to do this. 

### 7.3: The mapreduce function

A super-handy funtion is the mapreduce function.  It's a combination of mapping an array to another array and then reducing all in one.  Consider the following that takes an array, squares each element and then sums:

In [5]:
mapreduce(x->x^2,+,[1,2,3])

14

Since the function is the sum, this is also built into the sum command:

In [6]:
sum(x->x^2,[1,2,3])

14

This example, takes an array of strings, and find the average word length:

In [7]:
mapreduce(str->length(str),+,["This","is","a","very","boring","sentence"])/6

4.166666666666667

In [8]:
mapreduce(length, +,["This","is","a","very","boring","sentence"] )/6

4.166666666666667

### 7.4: Mapping a Function over an 2D array

If we have a 2D array, we can map over the rows or columns of the array using the `mapslices` function.  The result is a 1D array (vector).

In [9]:
A=[i+j for i=1:10,j=1:3]

10×3 Matrix{Int64}:
  2   3   4
  3   4   5
  4   5   6
  5   6   7
  6   7   8
  7   8   9
  8   9  10
  9  10  11
 10  11  12
 11  12  13

This is the column sums

In [10]:
mapslices(sum,A,dims=1)

1×3 Matrix{Int64}:
 65  75  85

And here are the row sums:

In [11]:
mapslices(sum,A,dims=2)

10×1 Matrix{Int64}:
  9
 12
 15
 18
 21
 24
 27
 30
 33
 36

### Summary of Chapter 7: Functional Programming

* What is functional programming and how does it differ for non-functional programming
* What are anonymous functions and how are they used.
* Using the map function to get a new array from an old one.
* Using the reduce function to get a single value from an array.
* Using the map function on multiple arrays
* Using the `mapreduce` and `mapslices` 